# Building Database for crime reports
The goal of this project is to create a database named *crimes_db* with a table *boston_crimes* with appropriate datatypes for storing the data from the boston.csv file. We'll be creating the table inside a schema named *crimes*. We'll also create the *readonly* and *readwrite* groups with the appropriate privileges. Finally, we will also need to create one user for each of these groups.